In [1]:
##############
#K. David Roell CFPB 7/12/16
#Compiles a report of all quality edit failures for HMDA LAR data
#
##############
import json
import os
import pandas as pd
import psycopg2
#from macro_sql import Q076 
with open('quality_sql.json') as f: #FIXME change to quality_sql.json and refactor the rest of code
    edit_sql = json.load(f)
    
#conn = psycopg2.connect("dbname=hmdamaster user=roellk") #connect and return connection
#cur = conn.cursor()#instantiate cursor object to use in SQL queries
#parameter format for local use #consider changing hmdamaster to roellk if db changes cause a fail
params = {
'dbname':'hmdamaster',
'user':'roellk',
'password':'',
'host':'localhost',

}

#connect_string = "dbname=%s user=%s host=%s password =%s" %(dbname, user, host, password) #set a string for connection to SQL
try:
    conn = psycopg2.connect(**params)
    cur = conn.cursor()
    print("i'm connected")

except psycopg2.Error as e: #if database connection results in an error print the following
    print("I am unable to connect to the database: ", e)


lar_tables = ['hmdalar2014', 'hmdalar2013']
#edit_list = [item for item in dir(SQL_text) if not item.startswith("__")]

i'm connected


In [2]:
def edit_text(table='hmdalar2014', edit_sql=edit_sql[0]['text'], edit_name=edit_sql[0]['name'], add_fields=[]):
    base_sql = """SELECT year, agency, COUNT(sequence) AS {edit_name} """.format(edit_name=edit_name)
    table_sql = "\nFROM {table}\n".format(table=table) 
    group_sql = """ GROUP BY year, agency"""
    for field in add_fields:
        base_sql = base_sql + ',' + field
        group_sql = group_sql + ', '+ field
    return_sql = base_sql + table_sql + edit_sql + group_sql + ';'
    print(return_sql)
    return return_sql 

In [ ]:
first = True
fields=['RID']
for edit in edit_sql:
    name, text = edit['name'], edit['text']
    print(name)
    #try:
    cur.execute(edit_text(edit_sql=text, edit_name=name, add_fields=fields))        
    data_df = pd.DataFrame(cur.fetchall())
    col_names = [desc[0] for desc in cur.description]
    data_df.columns = col_names
    print(data_df, '\n')
    if first:
        quality_df = data_df.copy()
        first = False
    else:
        quality_df=quality_df.merge(data_df, how='outer', on=['agency', 'year'])
        print(quality_df)

   # except:
    #    print('no dataz')
quality_df.to_csv('quality_edits.csv', index=False)

Q035
SELECT year, agency, COUNT(sequence) AS Q035 ,RID
FROM hmdalar2014
WHERE loan_type != '1' AND purchaser IN  ('1','3') GROUP BY year, agency, RID;
     year agency  q035         rid
0    2014      3     3  0000026967
1    2014      5     3  0000062542
2    2014      1     2  0000014062
3    2014      2     2  0000456157
4    2014      2     2  0000529958
5    2014      5     1  0000024438
6    2014      3     1  03-0365685
7    2014      3     6  0000014445
8    2014      9     1  0000012311
9    2014      3     1  0000030401
10   2014      3     5  0000017749
11   2014      1    19  0000009408
12   2014      7     7  7673900008
13   2014      1     8  0000703767
14   2014      3    38  0000005019
15   2014      2    22  0000058243
16   2014      9     1  99-0032158
17   2014      3    23  38-3548886
18   2014      1     9  0000000446
19   2014      3    10  0000017514
20   2014      3     1  0000019234
21   2014      7     1  5183509992
22   2014      3    10  0000012165
23   2014

In [ ]:
print(Q076['text'])